In [181]:
from pyspark.sql import SparkSession, functions
from pyspark.sql import functions as f
from minio.commonconfig import CopySource  
import requests
spark = SparkSession.builder.appName('load').enableHiveSupport().getOrCreate()

In [182]:
from minio import Minio
cliente = Minio("minio:9000",
access_key = "datalake",
secret_key = "datalake",
secure=False
)

In [185]:


objetos = cliente.list_objects("raw", recursive=True)
objetos_filtered= [objeto for objeto in objetos if objeto.object_name.startswith("api/TransSP/posicao")]

for objeto in objetos_filtered:
    df = spark.read.format("json").load("s3a://raw/"+objeto.object_name)
    df_tratado = df.select(explode(col("l")).alias("l_exploded")).select("l_exploded.c","l_exploded.cl","l_exploded.sl", "l_exploded.lt0", "l_exploded.lt1", "l_exploded.qv",explode(col("l_exploded.vs")).alias("vs_exploded"))
    df_normalizado =  df_tratado.select("c", "cl", "sl", "lt0", "lt1", "qv", "vs_exploded.a", "vs_exploded.is", "vs_exploded.p", "vs_exploded.px", "vs_exploded.py", "vs_exploded.sv", "vs_exploded.ta")
    df_normalizado.write.mode("append").parquet("s3a://trusted/posicao/")
    result = cliente.copy_object(  
    "raw",
    "api/TransSP/processed"+objeto.object_name[11:],  
    CopySource("raw", objeto.object_name)
    )
    cliente.remove_object("raw", objeto.object_name)